In [ ]:
%load_ext autoreload
%autoreload 2

# Experimenting with ``slfractals``

Some initialisations:

In [ ]:
import slfractals as slf
from matplotlib import pyplot as plt
from multiprocessing import Pool
from time import time

xlim = (-1.8, 1.)
ylim = (-0.9, 0.9)

## Generating a grid

In [ ]:
C1 = slf.get_grid(xlim, ylim, resw=10)
# display(C1)
C2 = slf.get_grid(xlim, ylim, resw=10, ratio="28:16")
# display(C2)
C = slf.get_grid(xlim, ylim, resw=2560)

## Computing

In [ ]:
comp = slf.Compute(slf.mandel, max_value=5, max_iter=300)

All at once:

In [ ]:
start = time()
grad, _ = comp(C.flatten())
grad1 = grad.reshape(C.shape)
print("Elapsed time: {}s".format(time() - start))

In [ ]:
%matplotlib inline
plt.imshow(grad1)

Split and join, serially:

In [ ]:
start = time()
spl = slf.split(C.flatten(), nportions=4)
grads, comps2 = zip(*map(comp, spl))
grad2 = slf.join_shape(grads, *C.shape)
print("Elapsed time: {}s".format(time() - start))

In [ ]:
%matplotlib inline
plt.imshow(grad2)

Split and join, with ``Pool``:

In [ ]:
start = time()
with Pool(2) as p:
    res = p.map(comp, slf.split(C.flatten(), nportions=4))
grads, comps3 = zip(*res)
grad3 = slf.join_shape(grads, *C.shape)
print("Elapsed time: {}s".format(time() - start))

In [ ]:
%matplotlib inline
plt.imshow(grad3)

Generating a new compute object for every batch:

In [ ]:
start = time()
with Pool(2) as p:
    res = p.map(slf.Compute(slf.mandel, max_value=5, max_iter=300), slf.split(C.flatten(), nportions=4))
grads, comps4 = zip(*res)
grad4 = slf.join_shape(grads, *C.shape)
print("Elapsed time: {}s".format(time() - start))

In [ ]:
%matplotlib inline
plt.imshow(grad4)

* To compute in parallel pool, the compute object is pickled and thus copied to each separate thread, resulting in a different memory id compared to the original (``comp3``)
* If we create a new compute object directly in ``Pool.map``, then of course we have another id from the beginning. (``comp4``)

In [ ]:
print("Original:  {}".format(id(comp)))
print("comps2  : {}".format(list(map(id, comps2))))
print("comps3  : {}".format(list(map(id, comps3))))
print("comps4  : {}".format(list(map(id, comps4))))

Functions to compute serially or in parallel:

In [ ]:
start = time()
grad_serial = slf.serial_compute(
    slf.mandel,
    C,
    max_iter=300,
    max_value=5,
    colorexp=2
)
print("Serial time: {}s".format(time() - start))

In [ ]:
start = time()
grad_parallel = slf.parallel_compute(
    slf.mandel,
    C,
    max_iter=300,
    max_value=5,
    colorexp=2,
    nproc=2
)
print("Serial time: {}s".format(time() - start))

In [ ]:
%matplotlib inline
ax1 = plt.subplot(121)
ax1.imshow(grad_serial)
ax2 = plt.subplot(122)
ax2.imshow(grad_parallel)